In [8]:
import os
from dotenv import load_dotenv
import openai
import csv
import pandas as pd
from openai import OpenAI
import json
import instaloader
from datetime import datetime
from itertools import dropwhile, takewhile

In [3]:
# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("No OpenAI API key found. Check your .env file.")

openai.api_key = openai_api_key

In [13]:
class GetInstagramProfile():
    def __init__(self) -> None:
        # Initializes the Instaloader object for downloading data from Instagram.
        self.L = instaloader.Instaloader()

    def download_users_profile_picture(self, username):
        # Downloads the profile picture of the specified username.
        self.L.download_profile(username, profile_pic_only=True)

    def download_users_posts_with_periods(self, username):
        # Downloads posts of a user within a specific date range.
        posts = instaloader.Profile.from_username(self.L.context, username).get_posts()
        SINCE = datetime(2034, 2, 1)  # Start date
        UNTIL = datetime(2024, 2, 14)  # End date

        # Filters posts within the specified date range and downloads them.
        for post in takewhile(lambda p: p.date > SINCE, dropwhile(lambda p: p.date > UNTIL, posts)):
            self.L.download_post(post, username)

    def download_hastag_posts(self, hashtag):
        # Downloads posts associated with a specific hashtag.
        for post in instaloader.Hashtag.from_name(self.L.context, hashtag).get_posts():
            self.L.download_post(post, target='#'+hashtag)

    def get_users_followers(self, user_name):
        '''Note: login required to get a profile's followers.'''
        # Prompts the user to login for accessing follower data.
        self.L.login(input("input your username: "), input("input your password: "))
        profile = instaloader.Profile.from_username(self.L.context, user_name)
        file = open("follower_names.txt", "a+")
        for followee in profile.get_followers():
            username = followee.username
            file.write(username + "\n")
            print(username)

    def get_users_followings(self, user_name):
        '''Note: login required to get a profile's followings.'''
        # Similar to get_users_followers, but for retrieving who a user is following.
        self.L.login(input("input your username: "), input("input your password: "))
        profile = instaloader.Profile.from_username(self.L.context, user_name)
        file = open("following_names.txt", "a+")
        for followee in profile.get_followees():
            username = followee.username
            file.write(username + "\n")
            print(username)

    def get_post_comments(self, username):
        # Retrieves and prints comments for each post of a specified user.
        posts = instaloader.Profile.from_username(self.L.context, username).get_posts()
        for post in posts:
            for comment in post.get_comments():
                print("comment.id  : "+str(comment.id))
                print("comment.owner.username  : "+comment.owner.username)
                print("comment.text  : "+comment.text)
                print("comment.created_at_utc  : "+str(comment.created_at_utc))
                print("************************************************")

    def get_post_info_csv(self, username):
        # Extracts detailed information about each post from a specified user and saves it into a CSV file.
        with open(username+'.csv', 'w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            posts = instaloader.Profile.from_username(self.L.context, username).get_posts()
            for post in posts:
                posturl = "https://www.instagram.com/p/"+post.shortcode
                writer.writerow(["post", post.mediaid, post.profile, post.caption, post.date, post.location, posturl, post.typename, post.mediacount, post.caption_hashtags, post.caption_mentions, post.tagged_users, post.likes, post.comments, post.title, post.url])
                
                for comment in post.get_comments():
                    writer.writerow(["comment", comment.id, comment.owner.username, comment.text, comment.created_at_utc])

if __name__ == "__main__":
    cls = GetInstagramProfile()
    # Uncomment the method you want to run. Make sure to provide the correct usernames or hashtags.
    cls.download_users_profile_picture("zuckerjagdwurst")
    cls.download_users_posts_with_periods("zuckerjagdwurst")
    cls.download_hastag_posts("zuckerjagdwurst")
    # cls.get_users_followers("zuckerjagdwurst")
    # cls.get_users_followings("zuckerjagdwurst")
    cls.get_post_comments("zuckerjagdwurst")
    cls.get_post_info_csv("zuckerjagdwurst")

fiarra_maureen/2015-10-10_15-07-14_UTC_profile_pic.jpg already exists


JSON Query to explore/tags/fiarra_maureen/: 404 Not Found [retrying; skip with ^C]
JSON Query to explore/tags/fiarra_maureen/: 404 Not Found [retrying; skip with ^C]


QueryReturnedNotFoundException: JSON Query to explore/tags/fiarra_maureen/: 404 Not Found